In [2]:
import pandas as pd
import requests as r
import time
import json
from mailjet_rest import Client
import os


In [3]:
inputs = pd.read_csv('server_input.csv')
patient = 'Susan Foster'
patient_email_addr = 'susan@foster.com'

The following method authenticate a third party tool called mailjet to set alert email to emergency contact of a person who just occured a fall. The initialization of this service needs a mailject account. For security reason, this will be provided using a separate email. 

In [4]:
def authenticate_mailjet():
    if os.getenv('MAILJET_PUBLIC_KEY') is not None \
    and os.getenv('MAILJET_SECRET_KEY') is not None:
        api_key = os.environ['MAILJET_PUBLIC_KEY']
        api_secret = os.environ['MAILJET_SECRET_KEY']
        mailjet = Client(auth=(api_key, api_secret), version='v3')
        return mailjet
    print('Please set MAILJET_PUBLIC_KEY and MAILJET_SECRET_KEY env variable before send email')
    return None
def send_email():
    mailjet = authenticate_mailjet()
    email = {
        'FromName': 'Fall Detection App',
        'FromEmail': 'sfu73318@gmail.com',
        'Subject': patient+' is in danger',
        'Text-Part': 'Hey! We suspect,'+patient+' just encountered a fall, please check.',
        'Recipients': [{'Email': patient_email_addr}]
    }
    if mailjet is not None:
        mailjet.send.create(email)

The function below will sequentially call our pretrained model in hosted in AWS. To get real time prediction. It will RESTful api for communiticating. The API is currently live and can be reached using the following URL. 'https://mdrmuhaimin.me/fall-detection/api/v1/predict'
Note:
* The server deployed in AWS can be run locally using the script [run_local_server.sh](run_local_server.sh) the file needed to run it is called [server.py](server.py)
* A sample restful call using CURL to our AWS server can be made by running the file [sample_local_server_rest_call.sh](sample_local_server_rest_call.sh)
* A sample restful call using CURL to our AWS server can be made by running the file [sample_aws_rest_call.sh](sample_aws_rest_call.sh)

In [10]:
i = 0
while (i < 100):
    i += 1
    time.sleep(1)
    request_data = json.dumps({
        "features":list(inputs.sample(n = 1).values.tolist())
    })
    headers = {'Content-Type': 'application/json'}
    response = r.post('https://mdrmuhaimin.me/fall-detection/api/v1/predict', headers=headers, data=request_data).json()
    prediction = response['prediction']
    print(request_data, prediction)
    if 1 in prediction:
        print('Fall detected for patient ', patient, 'Sending email to ', patient_email_addr)
        send_email()
        break

{"features": [[4.631069749999999, 0.0002342410521587419, 0.001351438238108884, -0.783035578125, 9.799279249514065, 0.001071265134832397, -8.600238953125, 0.0013263610914541206]]} [1]
Fall detected for patient  Susan Foster Sending email to  susan@foster.com
Please set MAILJET_PUBLIC_KEY and MAILJET_SECRET_KEY env variable before send email
